In [2]:
import csv

# Open the txt file
with open("tracking_just_vedan.txt", "r") as file:
    csv_reader = csv.reader(file, delimiter=" ")

    # Create a new list to store the filtered rows
    filtered_rows = []

    # Iterate over each row in the file
    for row in csv_reader:
        # Check if the second column (pedestrian id) is 1
        if row[1] == "1.0":
            # If it is 1, append the row to the filtered_rows list
            filtered_rows.append(row)

# Open a new file to write the filtered rows
with open("processed_vedan.txt", "w") as file:
    # Write each filtered row to the new file
    for row in filtered_rows:
        file.write(" ".join(row) + "\n")

In [1]:
import csv

# Open the txt file
with open("tracking_parallel.txt", "r") as file:
    csv_reader = csv.reader(file, delimiter=" ")

    # Create a new list to store the filtered rows
    filtered_rows = []

    # Iterate over each row in the file
    for row in csv_reader:
        # Only keep the first two pedestrians IDs since there were only
        # two pedestrians we kept track of and 1.0 and 2.0 were in all 41 frames
        if row[1] == "1.0" or row[1] == "2.0":
            # If it is 1, append the row to the filtered_rows list
            filtered_rows.append(row)

# Open a new file to write the filtered rows
with open("processed_parallel_tracking.txt", "w") as file:
    # Write each filtered row to the new file
    for row in filtered_rows:
        file.write(" ".join(row) + "\n")

In [3]:
import numpy as np

def inverse_project(H, loc):
    locHomogenous = np.hstack((loc, 1))
    locHomogenous = np.dot(H, locHomogenous)  # to image frame
    locXYZ = locHomogenous / locHomogenous[2]  # to pixels
    return locXYZ[:2]

# Load the homography matrix (assuming it's stored in 'H.txt')
H_iw = np.loadtxt('./H.txt')

# Open the input and output files
input_file = './processed_parallel.txt'
output_file = './processed_parallel_pixels.txt'

with open(input_file, 'r') as file:
    lines = file.readlines()

# Open the output file in write mode
with open(output_file, 'w') as file:
    for line in lines:
        # Split the line into columns
        columns = line.split()
        
        # Extract the x and y coordinates (assuming they are in the 4th to last and 2nd to last columns)
        x = float(columns[-4])
        y = float(columns[-2])
        
        # Convert the ground coordinates to pixel locations
        px, py = inverse_project(np.linalg.inv(H_iw), np.array([x, y]))
        
        # Update the columns with the pixel locations
        columns[-4] = str(px)
        columns[-2] = str(py)
        
        # Write the modified line to the output file
        file.write(' '.join(columns) + '\n')